In [226]:
# Importing modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [227]:
# Importing datasets from AWS
season_1516 = pd.read_csv('https://ids-storage-football-prediction.s3-eu-west-1.amazonaws.com/data/2015_2016.csv')
season_1617 = pd.read_csv('https://ids-storage-football-prediction.s3-eu-west-1.amazonaws.com/data/2016_2017.csv')
season_1718 = pd.read_csv('https://ids-storage-football-prediction.s3-eu-west-1.amazonaws.com/data/2017_2018.csv')
season_1819 = pd.read_csv('https://ids-storage-football-prediction.s3-eu-west-1.amazonaws.com/data/2018_2019.csv')
season_1920 = pd.read_csv('https://ids-storage-football-prediction.s3-eu-west-1.amazonaws.com/data/2019_2020.csv')

In [228]:
# Importing xG datasets
path = '../data'
years = [2015, 2016, 2017, 2018, 2019]
all_seasons = os.listdir(path)
all_teams = []
dct = {}
cnt = 0

for season in all_seasons:
    
    new_path = path + '/season_' + f'{years[cnt]}'
    all_teams = os.listdir(new_path)
    
    for team in all_teams:
        path_to_file = new_path + '/' + f'{team}'
        dct[team] = pd.read_csv(path_to_file, index_col=0)
    
    cnt+=1

In [229]:
# Example of usage
dct['Liverpool_2015.csv']

home_team             away_team   xG_home   xG_away
0   Liverpool           Bournemouth  2.150620  0.460805
1   Liverpool              West Ham  0.561737  0.972170
2   Liverpool               Norwich  1.904470  0.556271
3   Liverpool           Aston Villa  1.783310  0.892054
4   Liverpool           Southampton  0.682823  1.563830
5   Liverpool        Crystal Palace  2.156730  0.849281
6   Liverpool               Swansea  1.193580  0.367517
7   Liverpool  West Bromwich Albion  2.125830  0.835885
8   Liverpool             Leicester  1.379330  0.505167
9   Liverpool               Arsenal  1.162220  2.229120
10  Liverpool     Manchester United  0.952974  0.715703
11  Liverpool            Sunderland  1.577120  0.491843
12  Liverpool       Manchester City  1.167810  0.180318
13  Liverpool             Tottenham  1.379540  1.235790
14  Liverpool                 Stoke  2.214840  0.855080
15  Liverpool               Everton  3.244720  0.088633
16  Liverpool      Newcastle United  0.712881  0.253033
17  Liverpool               Watford  2.389190  0.930803
18  Liverpool               Chelsea  1.839340  0.812268

In [230]:
#Preprocessing datasets from AWS
#need to make a copy instead of working on original data
season_1516_ = season_1516.copy()
season_1617_ = season_1617.copy()
season_1718_ = season_1718.copy()
season_1819_ = season_1819.copy()
season_1920_ = season_1920.copy()

In [231]:
#Removing bet, referee and div, date columns from data
season_1516_ = season_1516_.loc[:, 'Date':'AR'].drop(['Referee'], axis = 1)
season_1617_ = season_1617_.loc[:, 'Date':'AR'].drop(['Referee'], axis = 1)
season_1718_ = season_1718_.loc[:, 'Date':'AR'].drop(['Referee'], axis = 1)
season_1819_ = season_1819_.loc[:, 'Date':'AR'].drop(['Referee'], axis = 1)
season_1920_ = season_1920_.loc[:, 'Date':'AR'].drop(['Referee'], axis = 1)
season_1516_.info() #20 columns left 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 21 columns):
Date        380 non-null object
HomeTeam    380 non-null object
AwayTeam    380 non-null object
FTHG        380 non-null int64
FTAG        380 non-null int64
FTR         380 non-null object
HTHG        380 non-null int64
HTAG        380 non-null int64
HTR         380 non-null object
HS          380 non-null int64
AS          380 non-null int64
HST         380 non-null int64
AST         380 non-null int64
HF          380 non-null int64
AF          380 non-null int64
HC          380 non-null int64
AC          380 non-null int64
HY          380 non-null int64
AY          380 non-null int64
HR          380 non-null int64
AR          380 non-null int64
dtypes: int64(16), object(5)
memory usage: 62.4+ KB


In [232]:
season_1516_['Date'] = pd.to_datetime(season_1516_['Date'], dayfirst = True) 
season_1617_['Date'] = pd.to_datetime(season_1617_['Date'], dayfirst = True)  
season_1718_['Date'] = pd.to_datetime(season_1718_['Date'], dayfirst = True)  
season_1819_['Date'] = pd.to_datetime(season_1819_['Date'], dayfirst = True) 
season_1920_['Date'] = pd.to_datetime(season_1920_['Date'], dayfirst = True) 

In [233]:
season_1516_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 21 columns):
Date        380 non-null datetime64[ns]
HomeTeam    380 non-null object
AwayTeam    380 non-null object
FTHG        380 non-null int64
FTAG        380 non-null int64
FTR         380 non-null object
HTHG        380 non-null int64
HTAG        380 non-null int64
HTR         380 non-null object
HS          380 non-null int64
AS          380 non-null int64
HST         380 non-null int64
AST         380 non-null int64
HF          380 non-null int64
AF          380 non-null int64
HC          380 non-null int64
AC          380 non-null int64
HY          380 non-null int64
AY          380 non-null int64
HR          380 non-null int64
AR          380 non-null int64
dtypes: datetime64[ns](1), int64(16), object(4)
memory usage: 62.4+ KB


In [234]:
#Changing team's names in each DataFrame in dict to be like in seasons, first copy
from copy import deepcopy
dct_ = deepcopy(dct) # working on copied dict with DataFrames

season_1516_teams_names_map = {'Arsenal' : 'Arsenal', 'Aston Villa' : 'Aston Villa', 'Bournemouth' : 'Bournemouth',
                               'Chelsea' : 'Chelsea', 'Crystal Palace' : 'Crystal Palace', 'Everton' : 'Everton',
                               'Leicester' : 'Leicester', 'Liverpool' : 'Liverpool', 'Manchester City' : 'Man City',
                               'Manchester United' : 'Man United', 'Newcastle United' : 'Newcastle', 'Norwich' : 'Norwich',
                               'Southampton' : 'Southampton', 'Stoke' : 'Stoke', 'Sunderland' : 'Sunderland',
                               'Swansea' : 'Swansea', 'Tottenham' : 'Tottenham', 'Watford' : 'Watford', 
                               'West Bromwich Albion' : 'West Brom', 'West Ham' : 'West Ham'}

season_1617_teams_names_map = {'Arsenal' : 'Arsenal',  'Bournemouth' : 'Bournemouth', 'Burnley' : 'Burnley', 
                               'Chelsea' : 'Chelsea', 'Crystal Palace' : 'Crystal Palace', 'Everton' : 'Everton',
                               'Leicester' : 'Leicester', 'Liverpool' : 'Liverpool', 'Manchester City' : 'Man City',
                               'Manchester United' : 'Man United',  'Middlesbrough' : 'Middlesbrough',
                               'Southampton' : 'Southampton', 'Stoke' : 'Stoke', 'Sunderland' : 'Sunderland',
                               'Swansea' : 'Swansea', 'Tottenham' : 'Tottenham', 'Watford' : 'Watford', 
                               'West Bromwich Albion' : 'West Brom', 'West Ham' : 'West Ham',  'Hull' : 'Hull'}

season_1718_teams_names_map = {'Arsenal' : 'Arsenal',  'Bournemouth' : 'Bournemouth', 'Burnley' : 'Burnley',
                               'Chelsea' : 'Chelsea', 'Crystal Palace' : 'Crystal Palace', 'Everton' : 'Everton',
                               'Leicester' : 'Leicester', 'Liverpool' : 'Liverpool', 'Manchester City' : 'Man City',
                               'Manchester United' : 'Man United',  'Newcastle United' : 'Newcastle', 'Brighton' : 'Brighton',
                               'Southampton' : 'Southampton', 'Stoke' : 'Stoke',  'Huddersfield' : 'Huddersfield',
                               'Swansea' : 'Swansea', 'Tottenham' : 'Tottenham', 'Watford' : 'Watford', 
                               'West Bromwich Albion' : 'West Brom', 'West Ham' : 'West Ham'}

season_1819_teams_names_map = {'Arsenal' : 'Arsenal',  'Bournemouth' : 'Bournemouth', 'Burnley' : 'Burnley',
                               'Chelsea' : 'Chelsea', 'Crystal Palace' : 'Crystal Palace', 'Everton' : 'Everton',
                               'Leicester' : 'Leicester', 'Liverpool' : 'Liverpool', 'Manchester City' : 'Man City',
                               'Manchester United' : 'Man United',  'Newcastle United' : 'Newcastle', 'Brighton' : 'Brighton',
                               'Southampton' : 'Southampton', 'Cardiff' : 'Cardiff',  'Huddersfield' : 'Huddersfield',
                               'Fulham' : 'Fulham', 'Tottenham' : 'Tottenham', 'Watford' : 'Watford', 
                               'Wolverhampton Wanderers' : 'Wolves', 'West Ham' : 'West Ham'}

season_1920_teams_names_map = {'Arsenal' : 'Arsenal',  'Bournemouth' : 'Bournemouth', 'Burnley' : 'Burnley',
                               'Chelsea' : 'Chelsea', 'Crystal Palace' : 'Crystal Palace', 'Everton' : 'Everton',
                               'Leicester' : 'Leicester', 'Liverpool' : 'Liverpool', 'Manchester City' : 'Man City',
                               'Manchester United' : 'Man United',  'Newcastle United' : 'Newcastle', 'Brighton' : 'Brighton',
                               'Southampton' : 'Southampton', 'Norwich' : 'Norwich',  'Sheffield United' : 'Sheffield United',
                               'Aston Villa' : 'Aston Villa', 'Tottenham' : 'Tottenham', 'Watford' : 'Watford', 
                               'Wolverhampton Wanderers' : 'Wolves', 'West Ham' : 'West Ham'}

season_map_list = [season_1516_teams_names_map, season_1617_teams_names_map ,season_1718_teams_names_map,
                  season_1819_teams_names_map,season_1920_teams_names_map]

idx = 0
for key, value in dct_.items():
    if f'{years[idx]}' in key:
        value['home_team'] = value['home_team'].map(season_map_list[idx])
        value['away_team'] = value['away_team'].map(season_map_list[idx])
        continue
    else:
        idx += 1
        value['home_team'] = value['home_team'].map(season_map_list[idx])
        value['away_team'] = value['away_team'].map(season_map_list[idx])

dct_['Man City_2015.csv'] = dct_.pop('Manchester City_2015.csv')
dct_['Man City_2016.csv'] = dct_.pop('Manchester City_2016.csv')
dct_['Man City_2017.csv'] = dct_.pop('Manchester City_2017.csv')
dct_['Man City_2018.csv'] = dct_.pop('Manchester City_2018.csv')
dct_['Man City_2019.csv'] = dct_.pop('Manchester City_2019.csv')
dct_['Man United_2015.csv'] = dct_.pop('Manchester United_2015.csv')
dct_['Man United_2016.csv'] = dct_.pop('Manchester United_2016.csv')
dct_['Man United_2017.csv'] = dct_.pop('Manchester United_2017.csv')
dct_['Man United_2018.csv'] = dct_.pop('Manchester United_2018.csv')
dct_['Man United_2019.csv'] = dct_.pop('Manchester United_2019.csv')
dct_['Newcastle_2015.csv'] = dct_.pop('Newcastle United_2015.csv')
dct_['Newcastle_2017.csv'] = dct_.pop('Newcastle United_2017.csv')
dct_['Newcastle_2018.csv'] = dct_.pop('Newcastle United_2018.csv')
dct_['Newcastle_2019.csv'] = dct_.pop('Newcastle United_2019.csv')
dct_['West Brom_2015.csv'] = dct_.pop('West Bromwich Albion_2015.csv')
dct_['West Brom_2016.csv'] = dct_.pop('West Bromwich Albion_2016.csv')
dct_['West Brom_2017.csv'] = dct_.pop('West Bromwich Albion_2017.csv')
dct_['Wolves_2018.csv'] = dct_.pop('Wolverhampton Wanderers_2018.csv')
dct_['Wolves_2019.csv'] = dct_.pop('Wolverhampton Wanderers_2019.csv')

In [235]:
#Attempt to merge datasets into one DataFrame...
#Creating DataFrames for each team in each season, then
#taking xG_home and xG_away from each team in each season from DICT,
#merging taken columns to sorted DataFrames for each team in each season,
#appending DataFrames to list, then created list adding to another list as SEASON...

list_of_lists_containing_data_frames = []
idx = 0
season_list = [season_1516_ ,season_1617_ ,season_1718_ ,season_1819_ ,season_1920_ ]
for season in season_list:
    list_of_data_frames_for_each_team_in_particular_season = []
    for h_team in season['HomeTeam'].sort_values().unique():
        df_sorted = season[season['HomeTeam'] == h_team].sort_values(by = ['AwayTeam'])
        xG_home_away_df = dct_[f'{h_team}_{years[idx]}.csv'].sort_values(by = ['away_team']).iloc[:,2:]
        xG_home_away_df = xG_home_away_df.reset_index().drop('index', axis = 1)
        df_sorted = df_sorted.reset_index().drop('index', axis = 1)

        df_sorted['xG_home'] = xG_home_away_df['xG_home']
        df_sorted['xG_away'] = xG_home_away_df['xG_away']
        df_ = df_sorted.set_index('Date')
        list_of_data_frames_for_each_team_in_particular_season.append(df_)
    idx += 1
    list_of_lists_containing_data_frames.append(list_of_data_frames_for_each_team_in_particular_season)


In [236]:
#Finally taking each list of seasons and creating one DataFrame for each season 
season_1516_ = pd.concat(list_of_lists_containing_data_frames[0]).sort_index()
season_1617_ = pd.concat(list_of_lists_containing_data_frames[1]).sort_index()
season_1718_ = pd.concat(list_of_lists_containing_data_frames[2]).sort_index()
season_1819_ = pd.concat(list_of_lists_containing_data_frames[3]).sort_index()
season_1920_ = pd.concat(list_of_lists_containing_data_frames[4]).sort_index()

In [237]:
# Now changing team's names to shortcuts e.g. Leicester : LEI 
season_1516_teams_names_map = {'Arsenal' : 'ARS', 'Aston Villa' : 'AVA', 'Bournemouth' : 'BOU',
                               'Chelsea' : 'CHE', 'Crystal Palace' : 'CRY', 'Everton' : 'EVE',
                               'Leicester' : 'LEI', 'Liverpool' : 'LIV', 'Man City' : 'MCI',
                               'Man United' : 'MUN', 'Newcastle' : 'NEW', 'Norwich' : 'NOR',
                               'Southampton' : 'SOU', 'Stoke' : 'STO', 'Sunderland' : 'SUN',
                               'Swansea' : 'SWA', 'Tottenham' : 'TOT', 'Watford' : 'WAT', 
                               'West Brom' : 'WBA', 'West Ham' : 'WHU'}

season_1617_teams_names_map = {'Arsenal' : 'ARS',  'Bournemouth' : 'BOU', 'Burnley' : 'BUR',
                               'Chelsea' : 'CHE', 'Crystal Palace' : 'CRY', 'Everton' : 'EVE',
                               'Hull' : 'HUL','Leicester' : 'LEI', 'Liverpool' : 'LIV', 'Man City' : 'MCI',
                               'Man United' : 'MUN',  'Middlesbrough' : 'MID',
                               'Southampton' : 'SOU', 'Stoke' : 'STO', 'Sunderland' : 'SUN',
                               'Swansea' : 'SWA', 'Tottenham' : 'TOT', 'Watford' : 'WAT', 
                               'West Brom' : 'WBA', 'West Ham' : 'WHU'}

season_1718_teams_names_map = {'Arsenal' : 'ARS',  'Bournemouth' : 'BOU', 'Burnley' : 'BUR',
                               'Chelsea' : 'CHE', 'Crystal Palace' : 'CRY', 'Everton' : 'EVE',
                               'Leicester' : 'LEI', 'Liverpool' : 'LIV', 'Man City' : 'MCI',
                               'Man United' : 'MUN',  'Newcastle' : 'NEW', 'Brighton' : 'BRI',
                               'Southampton' : 'SOU', 'Stoke' : 'STO',  'Huddersfield' : 'HUD',
                               'Swansea' : 'SWA', 'Tottenham' : 'TOT', 'Watford' : 'WAT', 
                               'West Brom' : 'WBA', 'West Ham' : 'WHU'}

season_1819_teams_names_map = {'Arsenal' : 'ARS',  'Bournemouth' : 'BOU', 'Burnley' : 'BUR',
                               'Chelsea' : 'CHE', 'Crystal Palace' : 'CRY', 'Everton' : 'EVE',
                               'Leicester' : 'LEI', 'Liverpool' : 'LIV', 'Man City' : 'MCI',
                               'Man United' : 'MUN',  'Newcastle' : 'NEW', 'Brighton' : 'BRI',
                               'Southampton' : 'SOU', 'Cardiff' : 'CAR',  'Huddersfield' : 'HUD',
                               'Fulham' : 'FUL', 'Tottenham' : 'TOT', 'Watford' : 'WAT', 
                               'Wolves' : 'WLV', 'West Ham' : 'WHU'}

season_1920_teams_names_map = {'Arsenal' : 'ARS',  'Bournemouth' : 'BOU', 'Burnley' : 'BUR',
                               'Chelsea' : 'CHE', 'Crystal Palace' : 'CRY', 'Everton' : 'EVE',
                               'Leicester' : 'LEI', 'Liverpool' : 'LIV', 'Man City' : 'MCI',
                               'Man United' : 'MUN',  'Newcastle' : 'NEW', 'Brighton' : 'BRI',
                               'Southampton' : 'SOU', 'Norwich' : 'NOR',  'Sheffield United' : 'SHU',
                               'Aston Villa' : 'AVA', 'Tottenham' : 'TOT', 'Watford' : 'WAT', 
                               'Wolves' : 'WLV', 'West Ham' : 'WHU'}

season_map_list = [season_1516_teams_names_map, season_1617_teams_names_map ,season_1718_teams_names_map,
                  season_1819_teams_names_map,season_1920_teams_names_map]
season_list = [season_1516_ ,season_1617_ ,season_1718_ ,season_1819_ ,season_1920_ ]

#now change...
idx = 0
for season in season_list:
    season['HomeTeam'] = season['HomeTeam'].map(season_map_list[idx])
    season['AwayTeam'] = season['AwayTeam'].map(season_map_list[idx])
    idx += 1

In [238]:
#Now remove date because it is will not help in further analysis
season_1516_ = season_1516_.reset_index(drop = True)
season_1617_ = season_1617_.reset_index(drop = True)
season_1718_ = season_1718_.reset_index(drop = True)
season_1819_ = season_1819_.reset_index(drop = True)
season_1920_ = season_1920_.reset_index(drop = True)

In [239]:
season_1516_.head(10)

HomeTeam AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  HS  AS  ...  HF  AF  HC  \
0      MUN      TOT     1     0   H     1     0   H   9   9  ...  12  12   1   
1      CHE      SWA     2     2   D     2     1   H  11  18  ...  15  16   4   
2      LEI      SUN     4     2   H     3     0   H  19  10  ...  13  17   6   
3      NOR      CRY     1     3   A     0     1   A  17  11  ...  14  20   1   
4      EVE      WAT     2     2   D     0     1   A  10  11  ...   7  13   8   
5      BOU      AVA     0     1   A     0     0   D  11   7  ...  13  13   6   
6      ARS      WHU     0     2   A     0     1   A  22   8  ...  12   9   5   
7      NEW      SOU     2     2   D     1     1   D   9  15  ...   9  12   6   
8      STO      LIV     0     1   A     0     0   D   7   8  ...   9  16   3   
9      WBA      MCI     0     3   A     0     2   A   9  19  ...  12   9   6   

   AC  HY  AY  HR  AR   xG_home   xG_away  
0   2   2   3   0   0  0.627539  0.674600  
1   8   1   3   1   0  0.643960  2.592030  
2   3   2   4   0   0  2.568030  1.459460  
3   4   1   0   0   0  1.130760  2.109750  
4   2   1   2   0   0  0.604226  0.557892  
5   3   3   4   0   0  0.876106  0.782253  
6   4   1   3   0   0  1.331660  0.535961  
7   6   2   4   0   0  1.546130  1.252900  
8   5   2   4   0   0  0.381274  0.329873  
9   6   4   1   0   0  0.435238  1.924200  

[10 rows x 22 columns]